# Kapitola 18: Bayesova veta - Matematicky kompas pro aktualizaci presvedceni

## Jak AI aktualizuje sve "nazory" na zaklade novych dukazu

---

### Co se naucite:
- Pochopit a implementovat Bayesovu vetu
- Rozlisovat Prior, Likelihood, Evidence a Posterior
- Odhalit past zakladni miry (Base Rate Fallacy)
- Aplikovat Bayesovske uvazovani na realne problemy
- Vytvorit jednoduchy spam filtr pomoci Naive Bayes

### Motivacni priklad:
Vas pritel ma suchy kasel. Je to nachladnuti? Chripka? Nebo vzacna nemoc "Drakonitida"?
Vas mozek intuitivne provadi **Bayesovske uvazovani** - vazi pravdepodobnost ruznych pricin
na zaklade novych dukazu. Dnes si tento proces matematicky formalizujeme.

---

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven pro Google Colab
!pip install matplotlib numpy pandas seaborn scikit-learn -q

print("Knihovny uspesne nainstalovany!")

In [ ]:
# Import knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import re

# Nastaveni vizualizaci
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Vsechny knihovny importovany!")
print("Tema: Bayesova veta a aktualizace presvedceni")

## 2. Bayesova veta - zakladni vzorec

Bayesova veta je elegantni vzorec, ktery spojuje ctyri klicove pojmy:

### Vzorec:
$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

### Interpretace:
- **P(A)** - **Prior** (apriorni pravdepodobnost): Co jsme verili PRED dukazem
- **P(B|A)** - **Likelihood** (verohodnost): Jak pravdepodobny je dukaz, pokud je A pravda
- **P(B)** - **Evidence** (dukaz): Celkova pravdepodobnost dukazu
- **P(A|B)** - **Posterior** (aposteriorni pravdepodobnost): Co verime PO dukazu

In [ ]:
def bayesova_veta(prior, likelihood, evidence):
    """
    Vypocita aposteriorni pravdepodobnost pomoci Bayesovy vety.
    
    Parametry:
    - prior: P(A) - apriorni pravdepodobnost hypotezy
    - likelihood: P(B|A) - pravdepodobnost dukazu pri platnosti hypotezy
    - evidence: P(B) - celkova pravdepodobnost dukazu
    
    Vraci:
    - posterior: P(A|B) - aposteriorni pravdepodobnost
    """
    if evidence == 0:
        raise ValueError("Evidence (P(B)) nemuze byt 0!")
    
    posterior = (likelihood * prior) / evidence
    return posterior


def vypocti_evidence(prior, likelihood, likelihood_neg):
    """
    Vypocita P(B) pomoci zakona uplne pravdepodobnosti.
    
    P(B) = P(B|A) * P(A) + P(B|~A) * P(~A)
    
    Parametry:
    - prior: P(A)
    - likelihood: P(B|A) - pravdepodobnost dukazu kdyz A plati
    - likelihood_neg: P(B|~A) - pravdepodobnost dukazu kdyz A neplati
    """
    evidence = likelihood * prior + likelihood_neg * (1 - prior)
    return evidence


print("Funkce pro Bayesovu vetu pripraveny!")
print("\nBayesova veta: P(A|B) = P(B|A) * P(A) / P(B)")

## 3. Priklad: Diagnoza Drakonitidy

Predstavte si vzacnou nemoc **Drakonitidu**:
- Postihuje pouze **0.1%** populace (velmi vzacna)
- **90%** nemocnych ma suchy kasel (hlavni priznak)
- Celkove kasle **5%** populace (z ruznych duvodu)

**Otazka:** Pokud nekdo kasle, jaka je pravdepodobnost, ze ma Drakonitidu?

In [ ]:
# Priklad: Diagnoza Drakonitidy
print("=" * 60)
print("PRIKLAD: Diagnoza vzacne nemoci Drakonitida")
print("=" * 60)

# Zadane hodnoty
p_drakonitida = 0.001        # P(A): Prior - 0.1% populace ma nemoc
p_kasel_pri_nemoci = 0.9     # P(B|A): Likelihood - 90% nemocnych kasle
p_kasel_celkem = 0.05        # P(B): Evidence - 5% populace kasle

print("\nZadane hodnoty:")
print(f"  P(Drakonitida) = {p_drakonitida*100:.2f}% (prior)")
print(f"  P(Kasel|Drakonitida) = {p_kasel_pri_nemoci*100:.1f}% (likelihood)")
print(f"  P(Kasel) = {p_kasel_celkem*100:.1f}% (evidence)")

# Vypocet pomoci Bayesovy vety
p_drakonitida_pri_kasli = bayesova_veta(
    prior=p_drakonitida,
    likelihood=p_kasel_pri_nemoci,
    evidence=p_kasel_celkem
)

print("\n" + "-" * 60)
print("VYSLEDEK:")
print("-" * 60)
print(f"\nP(Drakonitida|Kasel) = {p_drakonitida_pri_kasli*100:.2f}%")
print(f"\nInterpretace:")
print(f"  I kdyz ma pacient kasel (90% priznak nemoci),")
print(f"  pravdepodobnost ze ma Drakonitidu je pouze {p_drakonitida_pri_kasli*100:.2f}%!")

In [ ]:
# Vizualizace zmeny presvedceni
fig, ax = plt.subplots(figsize=(10, 6))

pravdepodobnosti = [p_drakonitida * 100, p_drakonitida_pri_kasli * 100]
nazvy = ['Pred dukazem\n(Prior)', 'Po dukazu\n(Posterior)']
barvy = ['#3498db', '#e74c3c']

bars = ax.bar(nazvy, pravdepodobnosti, color=barvy, edgecolor='black', linewidth=2)

# Popisky hodnot
for bar, prob in zip(bars, pravdepodobnosti):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{prob:.2f}%', ha='center', va='bottom', fontsize=14, fontweight='bold')

ax.set_ylabel('Pravdepodobnost Drakonitidy (%)', fontsize=12)
ax.set_title('Bayesova veta: Zmena presvedceni po zjisteni priznaku (kasel)', fontsize=14)
ax.set_ylim(0, max(pravdepodobnosti) * 1.3)

# Anotace
ax.annotate(f'Narust: {p_drakonitida_pri_kasli/p_drakonitida:.1f}x',
            xy=(0.5, max(pravdepodobnosti)/2), fontsize=12,
            ha='center', style='italic')

plt.tight_layout()
plt.show()

print(f"\nPravdepodobnost se zvysila {p_drakonitida_pri_kasli/p_drakonitida:.1f}x,")
print(f"ale stale je velmi nizka ({p_drakonitida_pri_kasli*100:.2f}%)!")

## 4. Base Rate Fallacy - Past zakladni miry

**Proc je vysledek tak prekvapive nizky?**

Toto je klasicky priklad **Base Rate Fallacy** (past zakladni miry):
- Nas mozek se soustredi na silny priznak (90% nemocnych kasle)
- A ignoruje klicovou informaci: nemoc je extremne vzacna (0.1%)

### Intuitivni vysvetleni:
V populaci 100,000 lidi:
- 100 ma Drakonitidu (0.1%), z nich 90 kasle
- 99,900 je zdravych, z nich ~5000 kasle (z jinych duvodu)
- Celkem kasle ~5090 lidi, ale jen 90 ma Drakonitidu!

In [ ]:
# Demonstrace Base Rate Fallacy vizualne
print("=" * 60)
print("BASE RATE FALLACY - Intuitivni vysvetleni")
print("=" * 60)

populace = 100000

# Vypocet skupin
nemocni = int(populace * p_drakonitida)  # 100
zdravi = populace - nemocni              # 99900

nemocni_kaslouci = int(nemocni * p_kasel_pri_nemoci)  # 90
nemocni_nekaslouci = nemocni - nemocni_kaslouci       # 10

# Kolik zdravych kasle? Musime dopocitat
# P(kasel) = P(kasel|nemoc)*P(nemoc) + P(kasel|zdrav)*P(zdrav)
# 0.05 = 0.9 * 0.001 + P(kasel|zdrav) * 0.999
p_kasel_pri_zdravi = (p_kasel_celkem - p_kasel_pri_nemoci * p_drakonitida) / (1 - p_drakonitida)

zdravi_kaslouci = int(zdravi * p_kasel_pri_zdravi)
zdravi_nekaslouci = zdravi - zdravi_kaslouci

celkem_kaslouci = nemocni_kaslouci + zdravi_kaslouci

print(f"\nV populaci {populace:,} lidi:")
print(f"\n  Nemocni (Drakonitida): {nemocni:,} ({nemocni/populace*100:.2f}%)")
print(f"    - Kaslouci nemocni: {nemocni_kaslouci}")
print(f"    - Nekaslouci nemocni: {nemocni_nekaslouci}")
print(f"\n  Zdravi: {zdravi:,} ({zdravi/populace*100:.2f}%)")
print(f"    - Kaslouci zdravi: {zdravi_kaslouci:,}")
print(f"    - Nekaslouci zdravi: {zdravi_nekaslouci:,}")
print(f"\n  CELKEM kaslouci: {celkem_kaslouci:,}")
print(f"    Z toho nemocnych: {nemocni_kaslouci} ({nemocni_kaslouci/celkem_kaslouci*100:.2f}%)")
print(f"\nKlicovy poznatek:")
print(f"  I kdyz 90% nemocnych kasle, vetsina kaslajicich je ZDRAVA!")

In [ ]:
# Vizualizace pomoci kolacovych grafu
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Graf 1: Cela populace
ax1 = axes[0]
sizes1 = [nemocni, zdravi]
labels1 = [f'Nemocni\n{nemocni:,}', f'Zdravi\n{zdravi:,}']
colors1 = ['#e74c3c', '#2ecc71']
ax1.pie(sizes1, labels=labels1, colors=colors1, autopct='%1.2f%%',
        startangle=90, explode=(0.05, 0))
ax1.set_title('Cela populace', fontsize=12, fontweight='bold')

# Graf 2: Vsichni kaslouci
ax2 = axes[1]
sizes2 = [nemocni_kaslouci, zdravi_kaslouci]
labels2 = [f'Nemocni kaslouci\n{nemocni_kaslouci}', f'Zdravi kaslouci\n{zdravi_kaslouci:,}']
colors2 = ['#e74c3c', '#2ecc71']
ax2.pie(sizes2, labels=labels2, colors=colors2, autopct='%1.2f%%',
        startangle=90, explode=(0.1, 0))
ax2.set_title('Pouze kaslouci lide', fontsize=12, fontweight='bold')

# Graf 3: Srovnani intuice vs realita
ax3 = axes[2]
categories = ['Intuice\n(90% priznak)', 'Realita\n(Bayes)']
values = [90, p_drakonitida_pri_kasli * 100]
colors3 = ['#9b59b6', '#3498db']
bars = ax3.bar(categories, values, color=colors3, edgecolor='black')
ax3.set_ylabel('Pravdepodobnost nemoci (%)')
ax3.set_title('Intuice vs Realita', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 100)
for bar, val in zip(bars, values):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 2,
             f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nBase Rate Fallacy: Nase intuice nas klame, protoze ignorujeme vzacnost nemoci!")

## 5. Interaktivni pruzkum - Jak Prior ovlivnuje Posterior

Pojdme prozkoumat, jak se meni nase presvedceni (posterior) v zavislosti na:
1. Vzacnosti nemoci (prior)
2. Sile priznaku (likelihood)

In [ ]:
def prozkoumej_bayese(priors, likelihood, likelihood_neg, nazev_hypotezy="Nemoc"):
    """
    Prozkouma jak se meni posterior pri ruznych hodnotach prior.
    """
    posteriors = []
    
    for prior in priors:
        evidence = vypocti_evidence(prior, likelihood, likelihood_neg)
        posterior = bayesova_veta(prior, likelihood, evidence)
        posteriors.append(posterior)
    
    return posteriors


# Ruzne hodnoty prior (vzacnost nemoci)
priors = np.linspace(0.001, 0.5, 100)

# Vypocet pro ruzne sily priznaku
likelihood_silny = 0.95   # Velmi specificky priznak
likelihood_stredni = 0.70 # Stredne specificky
likelihood_slaby = 0.40   # Slaby priznak

# Pravdepodobnost priznaku u zdravych (false positive rate)
false_positive = 0.05  # 5% zdravych ma taky tento priznak

posteriors_silny = prozkoumej_bayese(priors, likelihood_silny, false_positive)
posteriors_stredni = prozkoumej_bayese(priors, likelihood_stredni, false_positive)
posteriors_slaby = prozkoumej_bayese(priors, likelihood_slaby, false_positive)

# Vizualizace
plt.figure(figsize=(12, 6))

plt.plot(priors * 100, np.array(posteriors_silny) * 100, 
         'b-', linewidth=2, label=f'Silny priznak (95%)')
plt.plot(priors * 100, np.array(posteriors_stredni) * 100, 
         'g-', linewidth=2, label=f'Stredni priznak (70%)')
plt.plot(priors * 100, np.array(posteriors_slaby) * 100, 
         'r-', linewidth=2, label=f'Slaby priznak (40%)')

# Diagonala (bez dukazu by posterior = prior)
plt.plot(priors * 100, priors * 100, 'k--', alpha=0.5, label='Bez dukazu')

plt.xlabel('Prior - Prevalence nemoci (%)', fontsize=12)
plt.ylabel('Posterior - P(Nemoc|Priznak) (%)', fontsize=12)
plt.title('Jak vzacnost nemoci a sila priznaku ovlivnuji diagnozu', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim(0, 50)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

print("\nPoznatky:")
print("1. Pri vzacnych nemocich (nizky prior) je i silny priznak malo presvedcivy")
print("2. Sila priznaku (likelihood) ma vetsi vliv pri castejsich nemocich")
print("3. Prior (vzacnost) je kriticky faktor - neignorujte ho!")

## 6. Prakticka aplikace: Lekarsky test

Lekarsky test ma dve klicove vlastnosti:
- **Senzitivita**: P(Test+ | Nemoc) - jak dobre zachyti nemocne
- **Specificita**: P(Test- | Zdrav) - jak dobre identifikuje zdrave

I test s 99% senzitivitou a 99% specificitou muze byt zavadejici pri vzacne nemoci!

In [ ]:
def analyzuj_lekarsky_test(prevalence, senzitivita, specificita, nazev_testu="Test"):
    """
    Analyzuje lekarsky test pomoci Bayesovy vety.
    
    Parametry:
    - prevalence: P(Nemoc) - jak casta je nemoc v populaci
    - senzitivita: P(Test+|Nemoc) - true positive rate
    - specificita: P(Test-|Zdrav) - true negative rate
    """
    # Odvozene hodnoty
    p_nemoc = prevalence
    p_zdrav = 1 - prevalence
    
    p_pozitivni_nemocny = senzitivita      # True Positive Rate
    p_negativni_zdravy = specificita       # True Negative Rate
    p_pozitivni_zdravy = 1 - specificita   # False Positive Rate
    
    # P(Test+) - celkova pravdepodobnost pozitivniho testu
    p_pozitivni = (p_pozitivni_nemocny * p_nemoc + 
                   p_pozitivni_zdravy * p_zdrav)
    
    # P(Nemoc|Test+) - Bayesova veta
    p_nemoc_pri_pozitivnim = (p_pozitivni_nemocny * p_nemoc) / p_pozitivni
    
    # P(Zdrav|Test+) - false discovery rate
    p_zdrav_pri_pozitivnim = 1 - p_nemoc_pri_pozitivnim
    
    return {
        'nazev': nazev_testu,
        'prevalence': prevalence,
        'senzitivita': senzitivita,
        'specificita': specificita,
        'p_pozitivni': p_pozitivni,
        'ppv': p_nemoc_pri_pozitivnim,  # Positive Predictive Value
        'fdr': p_zdrav_pri_pozitivnim   # False Discovery Rate
    }


# Priklad: HIV test
print("=" * 60)
print("ANALYZA LEKARSKEHO TESTU")
print("=" * 60)

# Realisticke parametry HIV testu
vysledek = analyzuj_lekarsky_test(
    prevalence=0.001,    # 0.1% populace ma HIV
    senzitivita=0.999,   # 99.9% nemocnych zachyceno
    specificita=0.999,   # 99.9% zdravych spravne identifikovano
    nazev_testu="HIV Test"
)

print(f"\n{vysledek['nazev']}:")
print(f"  Prevalence: {vysledek['prevalence']*100:.2f}%")
print(f"  Senzitivita: {vysledek['senzitivita']*100:.1f}%")
print(f"  Specificita: {vysledek['specificita']*100:.1f}%")
print(f"\nVysledky:")
print(f"  P(Pozitivni test) = {vysledek['p_pozitivni']*100:.3f}%")
print(f"  P(Nemoc|Pozitivni) = {vysledek['ppv']*100:.1f}% (PPV)")
print(f"  P(Zdrav|Pozitivni) = {vysledek['fdr']*100:.1f}% (False Discovery)")
print(f"\n>>> I s 99.9% presnosti, {vysledek['fdr']*100:.1f}% pozitivnich vysledku jsou FALESNE!")

In [ ]:
# Srovnani testu pri ruznych prevalencich
prevalence_hodnoty = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]
nazvy_prevalence = ['0.01%', '0.1%', '1%', '5%', '10%', '20%']

ppv_hodnoty = []
for prev in prevalence_hodnoty:
    vysl = analyzuj_lekarsky_test(prev, 0.99, 0.99)
    ppv_hodnoty.append(vysl['ppv'] * 100)

# Vizualizace
plt.figure(figsize=(12, 6))

colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(prevalence_hodnoty)))
bars = plt.bar(nazvy_prevalence, ppv_hodnoty, color=colors, edgecolor='black')

plt.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='50% (nahoda)')

for bar, ppv in zip(bars, ppv_hodnoty):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
             f'{ppv:.1f}%', ha='center', fontsize=10, fontweight='bold')

plt.xlabel('Prevalence nemoci v populaci', fontsize=12)
plt.ylabel('PPV - Sance ze pozitivni test = nemoc (%)', fontsize=12)
plt.title('Test s 99% senzitivitou a 99% specificitou pri ruznych prevalencich', fontsize=13)
plt.legend()
plt.ylim(0, 105)

plt.tight_layout()
plt.show()

print("\nZaver:")
print("Pri vzacnych nemocich (nizka prevalence) i vynikajici testy davaji")
print("mnoho falesne pozitivnich vysledku. Proto se casto dela potvrzovaci test!")

## 7. Iterativni Bayes - Postupna aktualizace presvedceni

Jedna z nejsilnejsich vlastnosti Bayesovy vety je **iterativni aktualizace**:
- Dnesni posterior se stane zitrejsim prior
- S kazdym novym dukazem se nase presvedceni zpresni

In [ ]:
def iterativni_bayes(pocatecni_prior, dukazy, likelihood_pravda, likelihood_nepravda):
    """
    Postupne aktualizuje presvedceni na zaklade sekvence dukazu.
    
    Parametry:
    - pocatecni_prior: Pocatecni presvedceni P(H)
    - dukazy: Seznam dukazu (True = podporuje hypotezu, False = nepodporuje)
    - likelihood_pravda: P(D|H) - pravdepodobnost dukazu pokud hypoteza plati
    - likelihood_nepravda: P(D|~H) - pravdepodobnost dukazu pokud hypoteza neplati
    """
    historie = [pocatecni_prior]
    current_prior = pocatecni_prior
    
    for dukaz in dukazy:
        if dukaz:  # Dukaz podporuje hypotezu
            likelihood = likelihood_pravda
            likelihood_neg = likelihood_nepravda
        else:  # Dukaz nepodporuje hypotezu
            likelihood = 1 - likelihood_pravda
            likelihood_neg = 1 - likelihood_nepravda
        
        evidence = vypocti_evidence(current_prior, likelihood, likelihood_neg)
        posterior = bayesova_veta(current_prior, likelihood, evidence)
        
        historie.append(posterior)
        current_prior = posterior  # Posterior se stava novym prior
    
    return historie


# Priklad: Detekce podvodne mince
print("=" * 60)
print("ITERATIVNI BAYES: Detekce podvodne mince")
print("=" * 60)
print("\nScenario: Je mince fer (50/50) nebo podvodna (80% panna)?")

# Pocatecni presvedceni: 50% sance ze mince je podvodna
pocatecni_prior = 0.5

# Simulace hodu - radeji dostaneme 8 pannen z 10 hodu
np.random.seed(42)
# True = panna (podporuje hypotezu o podvodne minci)
hody = [True, True, False, True, True, True, False, True, True, True]

# Likelihood pro podvodnou minci
p_panna_podvodna = 0.8   # Podvodna mince = 80% panna
p_panna_fer = 0.5        # Fer mince = 50% panna

historie = iterativni_bayes(pocatecni_prior, hody, p_panna_podvodna, p_panna_fer)

print(f"\nSekvence hodu: {['Panna' if h else 'Orel' for h in hody]}")
print(f"\nVyvoj presvedceni (P(Podvodna mince)):")
for i, prob in enumerate(historie):
    if i == 0:
        print(f"  Start: {prob*100:.1f}%")
    else:
        hod = 'Panna' if hody[i-1] else 'Orel'
        print(f"  Po hodu {i} ({hod}): {prob*100:.1f}%")

In [ ]:
# Vizualizace vyvoje presvedceni
plt.figure(figsize=(12, 6))

x = range(len(historie))
y = [h * 100 for h in historie]

plt.plot(x, y, 'bo-', markersize=10, linewidth=2, label='P(Podvodna mince)')
plt.axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='Pocatecni presvedceni')

# Oznaceni jednotlivych hodu
for i in range(1, len(historie)):
    barva = 'green' if hody[i-1] else 'red'
    symbol = 'P' if hody[i-1] else 'O'
    plt.annotate(f'{symbol}', (i, y[i] + 3), ha='center', fontsize=12, color=barva, fontweight='bold')

plt.xlabel('Pocet hodu', fontsize=12)
plt.ylabel('Presvedceni o podvodne minci (%)', fontsize=12)
plt.title('Iterativni Bayesovska aktualizace: Detekce podvodne mince', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)
plt.xticks(x)

# Legenda pro symboly
plt.text(0.02, 0.98, 'P = Panna (podporuje podvod)\nO = Orel (proti podvodu)',
         transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nZavrecne presvedceni: {historie[-1]*100:.1f}% ze mince je podvodna")

## 8. Mini-projekt: Jednoduchy spam filtr (Naive Bayes)

Naive Bayes klasifikator je praktickou aplikaci Bayesovy vety.
Pouziva se napriklad pro:
- Spam filtry
- Analyzu sentimentu
- Kategorizaci dokumentu

In [ ]:
class JednoduckySpamFiltr:
    """
    Jednoduchy spam filtr zalozeny na Naive Bayes.
    """
    
    def __init__(self):
        self.spam_slova = defaultdict(int)
        self.ham_slova = defaultdict(int)  # ham = legitimni email
        self.spam_celkem = 0
        self.ham_celkem = 0
        self.pocet_spam = 0
        self.pocet_ham = 0
    
    def _tokenize(self, text):
        """Rozdeleni textu na slova."""
        text = text.lower()
        slova = re.findall(r'\b[a-z]+\b', text)
        return slova
    
    def trenuj(self, emaily, labely):
        """
        Natrenovani filtru na datach.
        
        Parametry:
        - emaily: seznam textu emailu
        - labely: seznam labelu (1 = spam, 0 = ham)
        """
        for email, label in zip(emaily, labely):
            slova = self._tokenize(email)
            
            if label == 1:  # Spam
                self.pocet_spam += 1
                for slovo in slova:
                    self.spam_slova[slovo] += 1
                    self.spam_celkem += 1
            else:  # Ham
                self.pocet_ham += 1
                for slovo in slova:
                    self.ham_slova[slovo] += 1
                    self.ham_celkem += 1
        
        print(f"Natrenovanao na {self.pocet_spam} spam a {self.pocet_ham} legitimnich emailech")
    
    def _p_slovo_trida(self, slovo, je_spam):
        """Vypocita P(slovo|trida) s Laplace smoothing."""
        if je_spam:
            return (self.spam_slova[slovo] + 1) / (self.spam_celkem + 2)
        else:
            return (self.ham_slova[slovo] + 1) / (self.ham_celkem + 2)
    
    def predikuj(self, email, verbose=False):
        """
        Predikuje zda je email spam.
        
        Vraci: (label, pravdepodobnost_spam)
        """
        slova = self._tokenize(email)
        
        # Prior pravdepodobnosti
        p_spam = self.pocet_spam / (self.pocet_spam + self.pocet_ham)
        p_ham = self.pocet_ham / (self.pocet_spam + self.pocet_ham)
        
        # Log pravdepodobnosti (pro numericku stabilitu)
        log_p_spam = np.log(p_spam)
        log_p_ham = np.log(p_ham)
        
        if verbose:
            print(f"\nAnalyza emailu: '{email[:50]}...'")
            print(f"Prior P(spam) = {p_spam:.2f}, P(ham) = {p_ham:.2f}")
        
        for slovo in slova:
            p_slovo_spam = self._p_slovo_trida(slovo, True)
            p_slovo_ham = self._p_slovo_trida(slovo, False)
            
            log_p_spam += np.log(p_slovo_spam)
            log_p_ham += np.log(p_slovo_ham)
        
        # Normalizace na pravdepodobnosti
        max_log = max(log_p_spam, log_p_ham)
        p_spam_norm = np.exp(log_p_spam - max_log)
        p_ham_norm = np.exp(log_p_ham - max_log)
        
        p_spam_final = p_spam_norm / (p_spam_norm + p_ham_norm)
        
        label = 1 if p_spam_final > 0.5 else 0
        
        if verbose:
            print(f"Finalni P(spam|email) = {p_spam_final:.2%}")
            print(f"Klasifikace: {'SPAM' if label == 1 else 'LEGITIMNI'}")
        
        return label, p_spam_final


print("Trida JednoduckySpamFiltr pripravena!")

In [ ]:
# Trenovaci data
trenovaci_emaily = [
    # Spam emaily
    "Congratulations you won million dollars click here now",
    "Free viagra pills buy now special offer",
    "You are selected winner claim your prize money",
    "Make money fast work from home opportunity",
    "Urgent your account will be suspended click verify",
    "Hot singles in your area click to meet",
    "Free gift card winner selected congratulations",
    "Lose weight fast miracle pill free trial",
    "Nigerian prince needs your help transfer money",
    "Click here for free iphone winner selected",
    # Legitimni emaily
    "Hi can we schedule meeting for tomorrow project discussion",
    "Please review the attached document and send feedback",
    "Thank you for your order confirmation number attached",
    "Meeting notes from today discussion action items listed",
    "Your flight booking confirmation details inside",
    "Project deadline reminder please submit by friday",
    "Team lunch tomorrow at noon please confirm attendance",
    "Quarterly report attached for your review",
    "Interview scheduled for next week details below",
    "Happy birthday wishes from the team"
]

trenovaci_labely = [1]*10 + [0]*10  # 1 = spam, 0 = ham

# Vytvoreni a trenovani filtru
filtr = JednoduckySpamFiltr()
filtr.trenuj(trenovaci_emaily, trenovaci_labely)

In [ ]:
# Testovani filtru
print("=" * 60)
print("TESTOVANI SPAM FILTRU")
print("=" * 60)

testovaci_emaily = [
    "You won free money click here to claim prize",
    "Meeting tomorrow at 3pm please confirm",
    "Congratulations winner selected for free gift",
    "Please review attached project proposal",
    "Make money fast working from home opportunity click now",
    "Thank you for attending the conference feedback welcome"
]

ocekavane = [1, 0, 1, 0, 1, 0]  # Ocekavane labely

spravne = 0
vysledky = []

for email, ocekavany in zip(testovaci_emaily, ocekavane):
    label, prob = filtr.predikuj(email)
    vysledky.append((email[:40], label, prob, ocekavany))
    if label == ocekavany:
        spravne += 1

# Zobrazeni vysledku
print(f"\n{'Email':<45} {'Predikce':<10} {'P(spam)':<10} {'Spravne'}")
print("-" * 75)
for email, label, prob, ocekavany in vysledky:
    predikce = "SPAM" if label == 1 else "HAM"
    spravnost = "OK" if label == ocekavany else "CHYBA"
    print(f"{email:<45} {predikce:<10} {prob:.2%}     {spravnost}")

print(f"\nPresnost: {spravne}/{len(testovaci_emaily)} = {spravne/len(testovaci_emaily)*100:.0f}%")

In [ ]:
# Vizualizace nejdulezitejsich spam slov
print("\n" + "=" * 60)
print("NEJDULEZITEJSI SPAM INDIKATORY")
print("=" * 60)

# Vypocet pomeru spam/ham pro kazde slovo
vsechna_slova = set(filtr.spam_slova.keys()) | set(filtr.ham_slova.keys())

spam_skore = {}
for slovo in vsechna_slova:
    p_spam = filtr._p_slovo_trida(slovo, True)
    p_ham = filtr._p_slovo_trida(slovo, False)
    # Pomer pravdepodobnosti (log odds)
    spam_skore[slovo] = np.log(p_spam / p_ham)

# Top spam a ham slova
serazena = sorted(spam_skore.items(), key=lambda x: x[1], reverse=True)
top_spam = serazena[:10]
top_ham = serazena[-10:][::-1]

# Vizualizace
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Spam slova
slova_spam = [s[0] for s in top_spam]
skore_spam = [s[1] for s in top_spam]
ax1.barh(slova_spam, skore_spam, color='#e74c3c')
ax1.set_xlabel('Log odds (vyssi = vice spam)')
ax1.set_title('Top 10 SPAM indikatory', fontweight='bold')
ax1.invert_yaxis()

# Ham slova
slova_ham = [s[0] for s in top_ham]
skore_ham = [-s[1] for s in top_ham]  # Inverze pro lepsi zobrazeni
ax2.barh(slova_ham, skore_ham, color='#2ecc71')
ax2.set_xlabel('Log odds (vyssi = vice legitimni)')
ax2.set_title('Top 10 LEGITIMNI indikatory', fontweight='bold')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

print("\nSlova jako 'free', 'winner', 'click' silne indikuji spam.")
print("Slova jako 'meeting', 'please', 'project' indikuji legitimni email.")

## 9. Shrnutí kapitoly

### Co jsme se naučili:

1. **Bayesova věta** je matematický nástroj pro aktualizaci přesvědčení:
   - P(A|B) = P(B|A) * P(A) / P(B)

2. **Čtyři klíčové pojmy**:
   - **Prior** - počáteční přesvědčení
   - **Likelihood** - síla důkazu
   - **Evidence** - celková pravděpodobnost důkazu
   - **Posterior** - aktualizované přesvědčení

3. **Base Rate Fallacy** - past, kdy ignorujeme vzácnost události

4. **Lékařské testy** - i velmi přesné testy mohou mít mnoho falešně pozitivních výsledků při vzácných nemocech

5. **Iterativní Bayes** - postupná aktualizace s každým novým důkazem

6. **Naive Bayes** - praktická aplikace pro klasifikaci (spam filtry, sentiment analýza)

### Bayesova věta v AI:
- Spam filtry
- Lékařská diagnostika
- Samořídící auta (fúze senzorů)
- Doporučovací systémy
- Zpracování přirozeného jazyka

In [ ]:
# Finalni interaktivni nastroj
def bayesuv_kalkulator():
    """Interaktivni Bayesuv kalkulator."""
    print("=" * 60)
    print("BAYESUV KALKULATOR")
    print("=" * 60)
    print("\nZadejte hodnoty pro vypocet P(Hypoteza|Dukaz):")
    
    # Preddefinovane hodnoty pro demonstraci
    scenare = {
        '1': {
            'nazev': 'Vzacna nemoc',
            'prior': 0.001,
            'likelihood': 0.95,
            'false_positive': 0.05
        },
        '2': {
            'nazev': 'Bezna nemoc',
            'prior': 0.10,
            'likelihood': 0.90,
            'false_positive': 0.10
        },
        '3': {
            'nazev': 'Spam detekce',
            'prior': 0.30,
            'likelihood': 0.85,
            'false_positive': 0.05
        }
    }
    
    print("\nVyberte scenar:")
    for key, val in scenare.items():
        print(f"  {key}. {val['nazev']}")
    
    # Pro demonstraci pouzijeme vsechny scenare
    print("\n" + "-" * 60)
    print("Vysledky pro vsechny scenare:")
    print("-" * 60)
    
    for key, scenar in scenare.items():
        evidence = vypocti_evidence(
            scenar['prior'], 
            scenar['likelihood'], 
            scenar['false_positive']
        )
        posterior = bayesova_veta(
            scenar['prior'], 
            scenar['likelihood'], 
            evidence
        )
        
        print(f"\n{scenar['nazev']}:")
        print(f"  Prior: {scenar['prior']*100:.1f}%")
        print(f"  Likelihood: {scenar['likelihood']*100:.0f}%")
        print(f"  False Positive Rate: {scenar['false_positive']*100:.0f}%")
        print(f"  >>> POSTERIOR: {posterior*100:.1f}%")

bayesuv_kalkulator()

## 10. Cviceni pro procviceni

### Cviceni 1: Podvodna mince
Mate minci, ktera muze byt ferova (50% panna) nebo podvodna (70% panna).
Po 5 hodech dostanete: Panna, Panna, Orel, Panna, Panna.
Jaka je pravdepodobnost, ze mince je podvodna? (Pocatecni prior = 50%)

### Cviceni 2: Druhy test
Lekarsky test ma senzitivitu 95% a specificitu 90%.
Nemoc ma prevalenci 2%.
Pokud prvni test je pozitivni a udelate druhy nezavisly test (take pozitivni),
jaka je finalni pravdepodobnost nemoci?

### Cviceni 3: Spam filtr
Rozsirte spam filtr o dalsi trenovaci data a otestujte na novych emailech.

In [ ]:
# Prostor pro vase reseni
print("Cviceni 1: Podvodna mince")
print("-" * 40)

# Vase reseni zde:
hody = [True, True, False, True, True]  # True = panna
historie = iterativni_bayes(0.5, hody, 0.7, 0.5)
print(f"Finalni pravdepodobnost: {historie[-1]*100:.1f}%")

In [ ]:
# Cviceni 2: Druhy test
print("Cviceni 2: Dva pozitivni testy")
print("-" * 40)

# Prvni test
vysledek1 = analyzuj_lekarsky_test(0.02, 0.95, 0.90)
print(f"Po prvnim testu: P(Nemoc) = {vysledek1['ppv']*100:.1f}%")

# Druhy test - prior je nyni posterior z prvniho testu
vysledek2 = analyzuj_lekarsky_test(vysledek1['ppv'], 0.95, 0.90)
print(f"Po druhem testu: P(Nemoc) = {vysledek2['ppv']*100:.1f}%")

---

## Dalsi kroky

V pristi kapitole se podivame na **Markovovy retezce** - dalsi dulezity koncept
pro modelovani sekvenci a pravdepodobnostnich prechodu v AI.

---

*Kapitola 18 - Bayesova veta | Kurz AI pro zacatecniky*